In [1]:
import pandas as pd

# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB
import re
import nltk
from nltk.corpus import stopwords

SEED = 3

In [11]:
df = pd.read_csv('../data/HEDOS.csv')
df.drop(columns=['coluna_combinada'], inplace=True)
df.fillna(0, inplace=True)

def replace_toxicity(label):
    if label == 'toxic':
        return 1
    elif label == 'not_toxic':
        return 0
    else:
        return 1  

# Aplicar a função nas colunas apropriadas
df['anotador1'] = df['anotador1'].apply(replace_toxicity).astype(int)
df['anotador2'] = df['anotador2'].apply(replace_toxicity).astype(int)
df['anotador3'] = df['anotador3'].apply(replace_toxicity).astype(int)
df['final_label'] = df['final_label'].apply(replace_toxicity).astype(int)

df

,text,anotador1,anotador2,anotador3,final_label
0,#BBB23 prova de bate volta chata e demorada De...,1,1,0,1
1,#BBB23 prova de sorte Puta merda Boa sorte pra...,0,0,1,0
2,"#BDRJ Fachel, tirando a coleta de lixo qual se...",0,0,1,0
3,#PaulistaoNaTNT que lixo de emissora não dever...,0,1,1,1
4,#askbsd tinha que ser sulista mesmo né vai a m...,1,1,1,1
...,...,...,...,...,...
5487,“odeio sulista” todos: simmm🥹😭🙏🏻😮‍💨🤮 “odeio no...,0,0,1,0
5488,👀🤗 é cada merda que eu vejo no Twitter não tem...,1,0,0,0
5489,👇👇👇👇esse cara é um FDP não sei como ainda não ...,1,1,1,1
5490,"🔥No Brasil que nós queremos, figuras como este...",0,1,1,1


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    # Remover IDs e padrões >>, mantendo expressões
    text = re.sub(r'>>\d+', '', text)
    # Remover menções como @usuário
    text = re.sub(r'@\w+', '', text)
    # Manter emojis, pontos de interrogação e exclamação
    text = re.sub(r'[^\w\s!?]', '', text)
    # Converter para minúsculas
    text = text.lower()
    return text

df['text_clean'] = df['text'].apply(preprocess_text)
df.drop(columns=['text'], inplace=True)
display(df)

,anotador1,anotador2,anotador3,final_label,text_clean
0,1,1,0,1,bbb23 prova de bate volta chata e demorada dep...
1,0,0,1,0,bbb23 prova de sorte puta merda boa sorte pra ...
2,0,0,1,0,bdrj fachel tirando a coleta de lixo qual serv...
3,0,1,1,1,paulistaonatnt que lixo de emissora não deveri...
4,1,1,1,1,askbsd tinha que ser sulista mesmo né vai a merda
...,...,...,...,...,...
5487,0,0,1,0,odeio sulista todos simmm odeio nordestino tod...
5488,1,0,0,0,é cada merda que eu vejo no twitter não tem como
5489,1,1,1,1,esse cara é um fdp não sei como ainda não mand...
5490,0,1,1,1,no brasil que nós queremos figuras como este d...


### **Exploratory Data Analysis (EDA)**

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5492 entries, 0 to 5491
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   anotador1    5492 non-null   int32 
 1   anotador2    5492 non-null   int32 
 2   anotador3    5492 non-null   int32 
 3   final_label  5492 non-null   int32 
 4   text_clean   5492 non-null   object
dtypes: int32(4), object(1)
memory usage: 128.8+ KB
None


In [14]:
print(df['final_label'].value_counts())

final_label
1    4302
0    1190
Name: count, dtype: int64


In [15]:
df.describe()

,anotador1,anotador2,anotador3,final_label
count,5492.000000,5492.000000,5492.000000,5492.000000
mean,0.569556,0.778587,0.852877,0.783321
std,0.495183,0.415236,0.354261,0.412019
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [16]:
df['text_clean']

0       bbb23 prova de bate volta chata e demorada dep...
1       bbb23 prova de sorte puta merda boa sorte pra ...
2       bdrj fachel tirando a coleta de lixo qual serv...
3       paulistaonatnt que lixo de emissora não deveri...
4       askbsd tinha que ser sulista mesmo né vai a merda
                              ...                        
5487    odeio sulista todos simmm odeio nordestino tod...
5488     é cada merda que eu vejo no twitter não tem como
5489    esse cara é um fdp não sei como ainda não mand...
5490    no brasil que nós queremos figuras como este d...
5491     vai poupar o nordestino de ouvir esse lixo de...
Name: text_clean, Length: 5492, dtype: object

In [17]:
df_anger = df[df['final_label'] == 1]
print(df_anger)

      anotador1  anotador2  anotador3  final_label  \
0             1          1          0            1   
3             0          1          1            1   
4             1          1          1            1   
6             0          1          1            1   
7             0          1          1            1   
...         ...        ...        ...          ...   
5484          0          1          1            1   
5485          0          1          1            1   
5489          1          1          1            1   
5490          0          1          1            1   
5491          1          1          1            1   

                                             text_clean  
0     bbb23 prova de bate volta chata e demorada dep...  
3     paulistaonatnt que lixo de emissora não deveri...  
4     askbsd tinha que ser sulista mesmo né vai a merda  
6     cariocanaband parabéns bandeirantes pela bela ...  
7     deixaonordestinovotar tomem vergonha na cara e...  
...

In [18]:
df_anger = df[df['final_label'] == 0]
print(df_anger)

      anotador1  anotador2  anotador3  final_label  \
1             0          0          1            0   
2             0          0          1            0   
5             0          0          1            0   
12            0          1          0            0   
16            0          0          0            0   
...         ...        ...        ...          ...   
5477          0          0          1            0   
5483          0          0          1            0   
5486          0          0          1            0   
5487          0          0          1            0   
5488          1          0          0            0   

                                             text_clean  
1     bbb23 prova de sorte puta merda boa sorte pra ...  
2     bdrj fachel tirando a coleta de lixo qual serv...  
5     bdrj o bom dia rj sempre foi ótimo mas depois ...  
12     lb 21 marcou pitico entregou  supercopa 22 ma...  
16    0147 da manhã e aqui estou eu morrendo de rir ...  
...

In [21]:
print(df.iloc[6][4], f'Hate.speech: {df.iloc[6][3]}', sep='\n')
print('\n')
print(df.iloc[12][4], f'Hate.speech: {df.iloc[12][3]}', sep='\n')

cariocanaband parabéns bandeirantes pela bela cobertura do campeonato carioca totalmente diferente do ano passado naquelas transmissões lixo da record
Hate.speech: 1


 lb 21 marcou pitico entregou  supercopa 22 marcou quatro pênaltis perdidos  carioca 22 marcou léo e ps já haviam entregado  supercopa 23 marcou dois time lixo e sistema def em dia de peneira nem noé carregou tanto animal como gabi em jogo de vice não merecia
Hate.speech: 0


C:\Users\laura\AppData\Local\Temp\ipykernel_16780\3587898699.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[6][4], f'Hate.speech: {df.iloc[6][3]}', sep='\n')
C:\Users\laura\AppData\Local\Temp\ipykernel_16780\3587898699.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[12][4], f'Hate.speech: {df.iloc[12][3]}', sep='\n')


### **Model training**

#### **Preprocessing**

In [22]:
X = df['text_clean']
y = df['final_label']

In [23]:
print(f'X nulls: {X.isna().sum()}')
print(f'y nulls: {y.isna().sum()}')

X nulls: 0
y nulls: 0


In [24]:
def get_pipe(model, params: dict):
    return Pipeline(steps=[
        ('vectorizer', TfidfVectorizer()),
        ('model', model(**params)),
    ])

def validate_pipe(pipe, X, y):
    accuracy = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
    precision = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='precision')
    recall = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='recall')
    f1 = cross_val_score(pipe, X, y, cv=StratifiedKFold(n_splits=5), scoring='f1')

    result_matrix = pd.DataFrame.from_dict({
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    })

    print(
        f'Accuracy: {result_matrix['accuracy'].mean():.1%}',
        f'Precision: {result_matrix['precision'].mean():.1%}',
        f'Recall: {result_matrix['recall'].mean():.1%}',
        f'F1: {result_matrix['f1'].mean():.1%}',
        sep='\n',
    )

    return result_matrix

In [25]:
multinomialnb_pipe = get_pipe(MultinomialNB, {})
validate_pipe(multinomialnb_pipe, X, y)

Accuracy: 78.3%
Precision: 78.3%
Recall: 100.0%
F1: 87.8%


,accuracy,precision,recall,f1
0,0.783439,0.783439,1.0,0.878571
1,0.783439,0.783439,1.0,0.878571
2,0.783242,0.783242,1.0,0.878447
3,0.783242,0.783242,1.0,0.878447
4,0.783242,0.783242,1.0,0.878447


In [26]:
complementnb_pipe = get_pipe(ComplementNB, {})
validate_pipe(complementnb_pipe, X, y)

Accuracy: 78.6%
Precision: 78.7%
Recall: 99.7%
F1: 88.0%


,accuracy,precision,recall,f1
0,0.784349,0.787293,0.993031,0.878274
1,0.787079,0.787351,0.997677,0.880123
2,0.788707,0.788073,0.998837,0.881026
3,0.782332,0.784081,0.996512,0.877624
4,0.788707,0.788603,0.997674,0.880903


In [27]:
bernoullinb_pipe = get_pipe(BernoulliNB, {})
validate_pipe(bernoullinb_pipe, X, y)

Accuracy: 78.9%
Precision: 79.9%
Recall: 97.5%
F1: 87.8%


,accuracy,precision,recall,f1
0,0.787989,0.803675,0.965157,0.877045
1,0.785259,0.796209,0.975610,0.876827
2,0.790528,0.800573,0.975581,0.879455
3,0.785974,0.796771,0.975581,0.877156
4,0.793260,0.798303,0.984884,0.881832
